In [28]:
import pmdarima as pm
import pandas as pd
import numpy as np
from pandas import datetime
from math import sqrt

import yfinance as yf
from skforecast.ForecasterAutoreg import ForecasterAutoreg
from skforecast.ForecasterAutoregMultiOutput import ForecasterAutoregMultiOutput
from catboost import CatBoostRegressor
from skforecast.model_selection import grid_search_forecaster
from skforecast.model_selection import backtesting_forecaster
import os
from joblib import dump, load

from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
import math

import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
from matplotlib import pyplot
import seaborn as sns

In [29]:
tickers=['BRK-A']

df = yf.download(tickers=tickers,start="2010-05-04", end = "2022-05-10", interval='1d')

[*********************100%***********************]  1 of 1 completed


In [30]:
for i in range(0,len(df.columns)):
    if (df == 0).any()[i]==True:
        df = df.drop(df.columns[i], axis=1)
df_x=df.iloc[:, 0:len(df.columns)].dropna()
i=len(df_x)

while i>0:
    df_x.iloc[i-1]=df_x.iloc[i-2]
    i=i-1
df_total=df_x
df_total=df_total.dropna()
l=20
train=df_total[:-l]
test=df_total[len(train):]

In [31]:
from pmdarima import auto_arima
import pmdarima as pm

In [32]:
# Fit a simple auto_arima model
arima = pm.auto_arima(train['Close'],
                      seasonal=True,
                      stationary=False,
                      trace=True,
                      )
pmd_predictions = arima.predict(n_periods=len(test['Close'][-l:]))
arima_mae = np.mean(np.abs(test['Close'][-l:] - pmd_predictions))
arima_rmse = (np.mean((test['Close'][-l:] - pmd_predictions)**2))**.5
arima_mape = np.sum(np.abs(pmd_predictions - test['Close'][-l:])) / (np.sum((np.abs(test['Close'][-l:]))))

Performing stepwise search to minimize aic
 ARIMA(2,2,2)(0,0,0)[0]             : AIC=inf, Time=1.44 sec
 ARIMA(0,2,0)(0,0,0)[0]             : AIC=62502.253, Time=0.04 sec
 ARIMA(1,2,0)(0,0,0)[0]             : AIC=59349.677, Time=0.10 sec
 ARIMA(0,2,1)(0,0,0)[0]             : AIC=inf, Time=0.30 sec
 ARIMA(2,2,0)(0,0,0)[0]             : AIC=58186.355, Time=0.15 sec
 ARIMA(3,2,0)(0,0,0)[0]             : AIC=57721.456, Time=0.29 sec
 ARIMA(4,2,0)(0,0,0)[0]             : AIC=57527.372, Time=0.36 sec
 ARIMA(5,2,0)(0,0,0)[0]             : AIC=57381.434, Time=0.56 sec
 ARIMA(5,2,1)(0,0,0)[0]             : AIC=inf, Time=2.36 sec
 ARIMA(4,2,1)(0,0,0)[0]             : AIC=inf, Time=2.86 sec
 ARIMA(5,2,0)(0,0,0)[0] intercept   : AIC=57383.426, Time=1.06 sec

Best model:  ARIMA(5,2,0)(0,0,0)[0]          
Total fit time: 9.542 seconds


In [33]:
pmd_predictions=pd.DataFrame(pmd_predictions)

In [34]:
predictions = pmd_predictions.set_index((pd.DataFrame(test['Close']).index))
result = pd.concat([predictions, pd.DataFrame(pd.DataFrame(test['Close']))], axis=1)
result.rename(columns = {0:'pred', 'Close':'y'}, inplace = True)

In [35]:
fig = px.line(result, y=result.columns[0:2], x=result.index)
fig.show()

In [36]:
MAE = mean_absolute_error(result['y'], result['pred'])
RSQUARED = r2_score(result['y'], result['pred'])
MAPE = mean_absolute_percentage_error(result['y'], result['pred'])
MSE = mean_squared_error(result['y'], result['pred'])
RMSE = math.sqrt(MSE)
COLUMN_NAMES = ["MSE", "RMSE", "MAE", "MAPE", "RSQUARED"]
values = np.array([f'{MSE:.2f}', f'{RMSE:.2f}', f'{MAE:.2f}', f'{MAPE:.2f}', f'{RSQUARED:.2f}'])

df = pd.DataFrame(values, columns=['Results'], index=COLUMN_NAMES)
df

,Results
MSE,246766635.71
RMSE,15708.81
MAE,12996.80
MAPE,0.03
RSQUARED,0.17
